## PROYECTO FINAL 
### UTP / MAESTRIA: ANALITICA DE DATOS
#### MATERIA: BIG DATA
##### ELABORADO POR: FRANCISCO D. TUDISCO

### INFORMACION SOBRE EL SERVIDOR

In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

# Optional: Set up authentication if needed
# auth_provider = PlainTextAuthProvider('your_username', 'your_password')
# cluster = Cluster(['127.0.0.1'], auth_provider=auth_provider)

# If no auth:
cluster = Cluster(['127.0.0.1'])

# Connect to session
session = cluster.connect('system')  # Replace with your keyspace name

# Execute a query
try:
    rows = session.execute('SELECT * FROM system.local;')

    print("Results:")
    for row in rows:
        print(row)

except Exception as e:
    print("Error during query:", e)

finally:
    cluster.shutdown()


### CREACION ESTRUCTURA DB CASSANDRA

In [ ]:
from cassandra.cluster import Cluster

KEYSPACE = "projectofinal"
TABLE = "wines_reviews"

# Connect to Cassandra
cluster = Cluster(["127.0.0.1"])
session = cluster.connect()

# Create keyspace if not exists
session.execute(f"""
    CREATE KEYSPACE IF NOT EXISTS {KEYSPACE}
    WITH REPLICATION = {{ 'class': 'SimpleStrategy', 'replication_factor': 1 }}
""")

# Use the keyspace
session.set_keyspace(KEYSPACE)

# Create table if not exists
session.execute(f"""
    CREATE TABLE IF NOT EXISTS {TABLE} (
        ID UUID PRIMARY KEY,
        country TEXT,
        description TEXT,
        designation TEXT,
        points DOUBLE,
        province TEXT,
        region_1 TEXT,
        region_2 TEXT,
        variety TEXT,
        winery TEXT,
        prices DOUBLE
)
""")

print("✅ Keyspace and table created successfully!")

cluster.shutdown()


### ELIMINAR TABLA

In [ ]:
from cassandra.cluster import Cluster

KEYSPACE = "projectofinal"
TABLE = "wines_reviews"

# Connect to Cassandra
cluster = Cluster(["127.0.0.1"])
session = cluster.connect()

# Use the keyspace
session.set_keyspace(KEYSPACE)

# Create table if not exists
session.execute(f"""
    DROP TABLE {TABLE}
""")

print("✅ table is droped successfully!")

cluster.shutdown()

### PYSPARK CASSANDRA 

In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

# Optional: Set up authentication if needed
# auth_provider = PlainTextAuthProvider('your_username', 'your_password')
# cluster = Cluster(['127.0.0.1'], auth_provider=auth_provider)

# If no auth:
cluster = Cluster(['127.0.0.1'])

# Connect to session
session = cluster.connect('projectofinal')  # Replace with your keyspace name

# Execute a query
try:
    rows = session.execute('DESCRIBE TABLES;')

    print("ENLISTA LAS TABLAS DEL KEYSPACE: 'projectofinal'")
    for row in rows:
        print(row)
    print()
except Exception as e:
    print("Error during query:", e)

finally:
    cluster.shutdown()

### DISPLAY DATOS: RESEÑA VINOS 

In [ ]:

from pyspark.sql.functions import *
from pyspark.sql.functions import regexp_extract
from pyspark.sql.functions import col
from pyspark.ml.stat import Correlation

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import pandas as pd 

# Optional: Set up authentication if needed
# auth_provider = PlainTextAuthProvider('your_username', 'your_password')
# cluster = Cluster(['127.0.0.1'], auth_provider=auth_provider)

# If no auth:
cluster = Cluster(['127.0.0.1'])

# Connect to session
session = cluster.connect('projectofinal')  # Replace with your keyspace name

# Execute a query
try:
    rows = session.execute('SELECT * FROM wines_reviews')

    print("Results:")
    for row in rows:
        print(row)
        df = pd.DataFrame(rows)
    print(df.head())
except Exception as e:
    print("Error during query:", e)

finally:
    cluster.shutdown()


### INSERCION DE DATOS

In [12]:
import uuid
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, format_number

# 1. Initialize Spark
spark = SparkSession.builder \
    .appName("TestApp") \
    .getOrCreate()

# 2. Connect to Cassandra
cluster = Cluster(['127.0.0.1'])
session = cluster.connect('projectofinal')


In [13]:
#datos #1 Reseñas de Vinos
data1 = spark.read.csv('/home/ubuntu/projectfinal_env/winemag-data-130k-v2.csv', header=True, 
                    inferSchema=False,quote='"',escape='"',multiLine=True) 
col=['country','description','designation','points','price','province','region_1','region_2','variety','winery']
df1_filtered=data1[col]
df1_filtered.head(3)

25/07/28 18:45:41 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, points, price, province, region_1, region_2, taster_name, taster_twitter_handle, title, variety, winery
 Schema: _c0, country, description, designation, points, price, province, region_1, region_2, taster_name, taster_twitter_handle, title, variety, winery
Expected: _c0 but found: 
CSV file: file:///home/ubuntu/projectfinal_env/winemag-data-130k-v2.csv


[Row(country='Italy', description="Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.", designation='Vulkà Bianco', points='87', price=None, province='Sicily & Sardinia', region_1='Etna', region_2=None, variety='White Blend', winery='Nicosia'),
 Row(country='Portugal', description="This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.", designation='Avidagos', points='87', price='15.0', province='Douro', region_1=None, region_2=None, variety='Portuguese Red', winery='Quinta dos Avidagos'),
 Row(country='US', description='Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel f

In [14]:
#datos #2 Reseñas de Vinos
data2=spark.read.csv('/home/ubuntu/projectfinal_env/winemag-data-150k.csv', header=True, 
                    inferSchema=False,quote='"',escape='"',multiLine=True) 
col=['country','description','designation','points','price','province','region_1','region_2','variety','winery']
df2_filtered=data2[col]
df2_filtered.head(3)

25/07/28 18:45:45 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, points, price, province, region_1, region_2, variety, winery
 Schema: _c0, country, description, designation, points, price, province, region_1, region_2, variety, winery
Expected: _c0 but found: 
CSV file: file:///home/ubuntu/projectfinal_env/winemag-data-150k.csv


[Row(country='US', description='This tremendous 100% varietal wine hails from Oakville and was aged over three years in oak. Juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. Enjoy 2022–2030.', designation="Martha's Vineyard", points='96', price='235.0', province='California', region_1='Napa Valley', region_2='Napa', variety='Cabernet Sauvignon', winery='Heitz'),
 Row(country='Spain', description='Ripe aromas of fig, blackberry and cassis are softened and sweetened by a slathering of oaky chocolate and vanilla. This is full, layered, intense and cushioned on the palate, with rich flavors of chocolaty black fruits and baking spices. A toasty, everlasting finish is heady but ideally balanced. Drink through 2023.', designation='Carodorum Selección Especial Reserva', points='96', price='110.0', pr

In [15]:
# Unificacion de los dataset
df =df1_filtered.union(df2_filtered)

df = df.withColumn("Prices", df['price'].cast("float"))
df = df.withColumn("Points", df['points'].cast("float"))

df = df.withColumn("Points", format_number("Points", 2))
df = df.withColumn("Prices", format_number("Prices", 2))
#df = df.drop("_c0",'price')
#df = df.withColumnRenamed("Prices", "prices")
#columna = ["country","description","designation","Points","prices","province","region_1","region_2","variety","winery"]
df= df.select(columna)

df.limit(5).toPandas().style.set_properties(**{
    'text-align': 'left',
    'white-space': 'pre-wrap'
}).set_table_styles([{
    'selector': 'th',
    'props': [('text-align', 'left')]
}])


25/07/28 18:45:47 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, points, price, province, region_1, region_2, taster_name, taster_twitter_handle, title, variety, winery
 Schema: _c0, country, description, designation, points, price, province, region_1, region_2, taster_name, taster_twitter_handle, title, variety, winery
Expected: _c0 but found: 
CSV file: file:///home/ubuntu/projectfinal_env/winemag-data-130k-v2.csv


,country,description,designation,Points,prices,province,region_1,region_2,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.",Vulkà Bianco,87.00,None,Sicily & Sardinia,Etna,None,White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's already drinkable, although it will certainly be better from 2016.",Avidagos,87.00,15.00,Douro,None,None,Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.",None,87.00,14.00,Oregon,Willamette Valley,Willamette Valley,Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.",Reserve Late Harvest,87.00,13.00,Michigan,Lake Michigan Shore,None,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it as a pleasantly unfussy country wine, it's a good companion to a hearty winter stew.",Vintner's Reserve Wild Child Block,87.00,65.00,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Sweet Cheeks


In [3]:

try:
    # 3. Load CSV into Spark DataFrame
    data1 = spark.read.csv(
        '/home/ubuntu/projectfinal_env/wines_reviews_final.csv',
        header=True,
        inferSchema=False,
        quote='"',
        escape='"',
        multiLine=True
    )

    # 4. Prepare insert statement for Cassandra
    insert_stmt = session.prepare("""
        INSERT INTO wines_reviews (
            id, country, description, designation,
            points, prices, province, region_1, region_2, variety, winery
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """)

    # 5. Iterate through DataFrame rows and insert into Cassandra
    for row in data1.collect():
        # Convert string to appropriate types
        try:
            points = float(row['Points']) if row['Points'] else None
            prices = float(row['Prices']) if row['Prices'] else None
        except:
            points = None
            prices = None

        session.execute(
            insert_stmt,
            (
                uuid.uuid4(),                      # id
                row['country'],                    # country
                row['description'],                # description
                row['designation'],                # designation
                points,                            # points
                prices,                            # prices
                row['province'],                   # province
                row['region_1'],                   # region_1
                row['region_2'],                   # region_2
                row['variety'],                    # variety
                row['winery']                      # winery
            )
        )

    print("✅ Inserción completada en Cassandra.")

except Exception as e:
    print("❌ Error during query or insertion:", e)

finally:
    cluster.shutdown()
    print("✅ Conexión a Cassandra cerrada.")


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/28 13:12:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/28 13:12:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/07/28 13:12:58 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, Points, price, province, region_1, region_2, variety, winery, Prices
 Schema: _c0, country, description, designation, Points, price, province, region_1, region_2, variety, winery, Prices
Expected: _c0 but found: 
CSV file: file:///home/ubuntu/projectfinal_env/wines_reviews_final.csv
                                                                                

✅ Inserción completada en Cassandra.
✅ Conexión a Cassandra cerrada.


#### Validacion

In [ ]:
    data1 = spark.read.csv(
        '/home/ubuntu/projectfinal_env/wines_reviews_final.csv',
        header=True,
        inferSchema=False,
        quote='"',
        escape='"',
        multiLine=True
    )
data1.toPandas().shape